In [1]:
import os
import random
import foolbox as fb
import numpy as np
import cv2 as cv


In [2]:
import tensorflow as tf
from tensorflow import keras
keras_model_path = "../outs/model"
model = tf.keras.models.load_model(keras_model_path)


In [3]:
fmodel = fb.TensorFlowModel(
    model, 
    bounds=(0, 255), 
    preprocessing=dict(flip_axis=-1, mean=[104.0, 116.0, 123.0])
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [4]:
fmodel = fmodel.transform_bounds((0, 1))
fmodel

In [5]:
data_dir = "../outs/comprs1/"

In [19]:

img_height = 112
img_width = 112*2
img = keras.preprocessing.image.load_img(
    f"{data_dir}/like/"+random.choice(os.listdir(f"{data_dir}/like")), target_size=(img_height, img_width)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

score


<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.7900544 , 0.20994562], dtype=float32)>

In [20]:

labels = tf.constant([0])
target_classes = tf.constant([1])
images = img_array

criterion = fb.criteria.TargetedMisclassification(target_classes)

attack = fb.attacks.L2CarliniWagnerAttack(steps=100)

advs, _, is_adv = attack(fmodel, images, criterion, epsilons=None)

print(
    fb.utils.accuracy(fmodel, images, labels),
    fb.utils.accuracy(fmodel, advs, labels),
    fb.utils.accuracy(fmodel, advs, target_classes)
)


1.0 1.0 0.0


In [21]:

img = np.transpose(advs[0].cpu().numpy(), (0,1,2)) * 255
cv.imwrite("../temp/bads.jpg", img)


True

In [22]:
np.max(advs)

0.0